# Notebook 03: Feature Engineering and Preprocessing

---

## Executive Summary

Feature engineering is often called the **"secret sauce"** of machine learning. Raw data, however clean, rarely maximizes model performance. This notebook transforms our cleaned dataset into a **feature matrix optimized for predictive modeling**.

### What This Notebook Covers:

1. **Feature Selection Rationale** — Choosing which features to include and why
2. **Target Variable Transformation** — Log transformation to address skewness
3. **Feature Creation** — Engineering new predictive features from existing data
4. **Categorical Encoding** — Converting text to numbers machine learning can use
5. **Numerical Scaling** — Standardizing feature ranges
6. **Final Feature Matrix** — Prepared data ready for modeling

---

## 1. Environment Setup and Data Loading

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================

# Core Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Statistical Libraries
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

# System
import os
import warnings
import joblib

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Environment configured successfully")
print(f"   Random State: {RANDOM_STATE}")

In [ ]:
# Path Configuration
BASE_DIR = os.path.dirname(os.getcwd())
DATA_PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
FIGURES_DIR = os.path.join(BASE_DIR, 'report', 'figures')
MODELS_DIR = os.path.join(BASE_DIR, 'models')

# Ensure directories exist
for directory in [DATA_PROCESSED_DIR, FIGURES_DIR, MODELS_DIR]:
    os.makedirs(directory, exist_ok=True)

# Load cleaned data
DATA_PATH = os.path.join(DATA_PROCESSED_DIR, 'cleaned_data.csv')

if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print(f"✅ Loaded cleaned data from: {DATA_PATH}")
else:
    # Fallback
    RAW_PATH = os.path.join(BASE_DIR, 'data', 'raw', 'WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv')
    df = pd.read_csv(RAW_PATH)
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    print(f"⚠️ Loaded raw data with basic cleaning.")

print(f"\n📊 Dataset: {len(df):,} rows × {len(df.columns)} columns")

---

## 2. Feature Selection Rationale

Before engineering features, we must decide **which raw features to use**. Not all available features are appropriate for modeling.

### 2.1 Feature Categories

| Category | Include? | Reason |
|----------|----------|--------|
| **Identifier (Customer ID)** | ❌ No | Not predictive; would cause overfitting |
| **Target Variable (CLV)** | Target | What we're predicting |
| **Dates** | ❌ No | Need special handling; extract features instead |
| **Demographics** | ✅ Yes | Predictive of customer behavior |
| **Policy Details** | ✅ Yes | Core business features |
| **Behavioral Data** | ✅ Yes | Strong signals of customer value |

In [ ]:
# Feature categorization
print("=" * 80)
print("FEATURE CATEGORIZATION")
print("=" * 80)

# Target variable
TARGET = 'customer_lifetime_value'

# Columns to exclude from features
EXCLUDE_COLS = [
    'customer',              # Identifier - not predictive
    'customer_lifetime_value',  # Target variable
    'effective_to_date',     # Date - needs special handling
]

# Get all feature columns
all_cols = df.columns.tolist()
feature_cols = [col for col in all_cols if col not in EXCLUDE_COLS]

print(f"\n📋 Total Columns: {len(all_cols)}")
print(f"📋 Excluded Columns: {EXCLUDE_COLS}")
print(f"📋 Feature Columns: {len(feature_cols)}")
print(f"\nFeatures to use: {feature_cols}")

In [ ]:
# Separate feature types
numeric_features = df[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df[feature_cols].select_dtypes(include=['object']).columns.tolist()

print(f"\n🔢 Numerical Features ({len(numeric_features)}):")
for f in numeric_features:
    print(f"   • {f}")

print(f"\n📋 Categorical Features ({len(categorical_features)}):")
for f in categorical_features:
    n_unique = df[f].nunique()
    print(f"   • {f} ({n_unique} categories)")

### 2.2 Multicollinearity Check (VIF)

**Variance Inflation Factor (VIF)** measures how much the variance of a regression coefficient is inflated due to collinearity. 

- VIF = 1: No correlation with other features
- VIF > 5: Moderate multicollinearity (concerning)
- VIF > 10: High multicollinearity (problematic)

In [ ]:
# Calculate VIF for numerical features
print("=" * 80)
print("MULTICOLLINEARITY CHECK (VIF)")
print("=" * 80)

def calculate_vif(df, features):
    """Calculate Variance Inflation Factor for each feature."""
    vif_data = pd.DataFrame()
    vif_data['Feature'] = features
    
    # Handle missing values and create feature matrix
    X = df[features].dropna()
    
    vif_values = []
    for i in range(len(features)):
        try:
            vif = variance_inflation_factor(X.values, i)
            vif_values.append(vif)
        except:
            vif_values.append(np.nan)
    
    vif_data['VIF'] = vif_values
    vif_data['Interpretation'] = vif_data['VIF'].apply(
        lambda x: 'Good' if x < 5 else ('Moderate' if x < 10 else 'High')
    )
    
    return vif_data.sort_values('VIF', ascending=False)

vif_results = calculate_vif(df, numeric_features)
print("\n📊 VIF Analysis Results:")
vif_results

---

## 3. Target Variable Transformation

From our EDA, we identified that CLV is **right-skewed**. While tree-based models are robust to skewness, transformation often improves performance and interpretability.

### Why Log Transformation?

The `log1p` transformation ($\ln(1+x)$) is preferred over `log` because:
1. It handles zero values (log(0) is undefined, but log(1) = 0)
2. It compresses the range of extreme values
3. It makes the distribution more Gaussian-like

In [ ]:
# Target transformation
print("=" * 80)
print("TARGET VARIABLE TRANSFORMATION")
print("=" * 80)

# Original distribution stats
print(f"\n📊 Original CLV Statistics:")
print(f"   Mean:     ${df[TARGET].mean():,.2f}")
print(f"   Median:   ${df[TARGET].median():,.2f}")
print(f"   Skewness: {df[TARGET].skew():.4f}")
print(f"   Kurtosis: {df[TARGET].kurtosis():.4f}")

# Apply log1p transformation
df['log_clv'] = np.log1p(df[TARGET])

print(f"\n📊 Log-Transformed CLV Statistics:")
print(f"   Mean:     {df['log_clv'].mean():.4f}")
print(f"   Median:   {df['log_clv'].median():.4f}")
print(f"   Skewness: {df['log_clv'].skew():.4f}")
print(f"   Kurtosis: {df['log_clv'].kurtosis():.4f}")

print(f"\n✅ Skewness reduced from {df[TARGET].skew():.2f} to {df['log_clv'].skew():.2f}")

In [ ]:
# Visualize transformation effect
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original
sns.histplot(df[TARGET], kde=True, ax=axes[0], color='steelblue', bins=50)
axes[0].set_title('Original CLV Distribution', fontweight='bold')
axes[0].set_xlabel('Customer Lifetime Value ($)')
axes[0].axvline(df[TARGET].mean(), color='red', linestyle='--', label=f'Mean')
axes[0].axvline(df[TARGET].median(), color='green', linestyle='-.', label=f'Median')
axes[0].legend()

# Log-transformed
sns.histplot(df['log_clv'], kde=True, ax=axes[1], color='seagreen', bins=50)
axes[1].set_title('Log-Transformed CLV Distribution', fontweight='bold')
axes[1].set_xlabel('log(1 + CLV)')
axes[1].axvline(df['log_clv'].mean(), color='red', linestyle='--', label=f'Mean')
axes[1].axvline(df['log_clv'].median(), color='green', linestyle='-.', label=f'Median')
axes[1].legend()

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_target_transformation.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 03_target_transformation.png")

---

## 4. Feature Engineering

We create new features based on **domain knowledge** and **statistical relationships** identified during EDA.

### 4.1 Interaction Features

Interaction features capture the **combined effect** of two or more features that may not be captured by the individual features alone.

In [ ]:
# Feature Engineering
print("=" * 80)
print("FEATURE ENGINEERING")
print("=" * 80)

# 1. Interaction Feature: Coverage × Education
# Rationale: Different education levels may have different risk profiles per coverage type
if 'coverage' in df.columns and 'education' in df.columns:
    df['coverage_education'] = df['coverage'] + '_' + df['education']
    print(f"\n✅ Created: coverage_education (Interaction feature)")
    print(f"   Unique combinations: {df['coverage_education'].nunique()}")

# 2. Insurance Loss Ratio (ILR)
# Rationale: Claims relative to premium is a standard insurance metric
if 'total_claim_amount' in df.columns and 'monthly_premium_auto' in df.columns:
    # Avoid division by zero
    df['insurance_loss_ratio'] = df['total_claim_amount'] / (df['monthly_premium_auto'] + 1)
    print(f"\n✅ Created: insurance_loss_ratio (Claims / Premium)")
    print(f"   Mean ILR: {df['insurance_loss_ratio'].mean():.2f}")

# 3. Premium per Policy
# Rationale: Average premium contribution per policy
if 'monthly_premium_auto' in df.columns and 'number_of_policies' in df.columns:
    df['premium_per_policy'] = df['monthly_premium_auto'] / (df['number_of_policies'] + 1)
    print(f"\n✅ Created: premium_per_policy (Premium / # Policies)")

# 4. Customer Engagement Score (Complaints + Response)
if 'number_of_open_complaints' in df.columns:
    # Lower is better (fewer complaints)
    df['complaint_flag'] = (df['number_of_open_complaints'] > 0).astype(int)
    print(f"\n✅ Created: complaint_flag (Binary: has complaints)")
    print(f"   Customers with complaints: {df['complaint_flag'].sum():,} ({df['complaint_flag'].mean()*100:.1f}%)")

# 5. Policy Tenure Category
if 'months_since_policy_inception' in df.columns:
    df['tenure_category'] = pd.cut(
        df['months_since_policy_inception'],
        bins=[0, 12, 36, 60, np.inf],
        labels=['new', 'established', 'loyal', 'veteran']
    )
    print(f"\n✅ Created: tenure_category (Binned tenure)")
    print(df['tenure_category'].value_counts())

In [ ]:
# Summary of engineered features
engineered_features = ['coverage_education', 'insurance_loss_ratio', 'premium_per_policy', 
                        'complaint_flag', 'tenure_category']
engineered_features = [f for f in engineered_features if f in df.columns]

print(f"\n📋 Summary of Engineered Features:")
for f in engineered_features:
    dtype = df[f].dtype
    print(f"   • {f}: {dtype}")

---

## 5. Prepare Final Feature Set

Now we define the final set of features to use for modeling.

In [ ]:
# Define final feature set
print("=" * 80)
print("FINAL FEATURE SET DEFINITION")
print("=" * 80)

# Columns to drop from features
drop_for_modeling = [
    'customer',                    # Identifier
    'customer_lifetime_value',     # Original target
    'log_clv',                     # Will be our target
    'effective_to_date',           # Date column
    'policy',                      # Redundant with policy_type
]

# Get final features
final_feature_cols = [col for col in df.columns if col not in drop_for_modeling]

# Separate by type
final_numeric = df[final_feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
final_categorical = df[final_feature_cols].select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n📊 Final Feature Count: {len(final_feature_cols)}")
print(f"   Numerical: {len(final_numeric)}")
print(f"   Categorical: {len(final_categorical)}")

print(f"\n🔢 Numerical Features:")
for f in final_numeric:
    print(f"   • {f}")

print(f"\n📋 Categorical Features:")
for f in final_categorical:
    print(f"   • {f} ({df[f].nunique()} categories)")

---

## 6. Train-Test Split

Before preprocessing, we split the data to prevent **data leakage**. The preprocessing (scaling, encoding) must be fit only on training data.

In [ ]:
# Prepare X and y
print("=" * 80)
print("TRAIN-TEST SPLIT")
print("=" * 80)

# Target
y = df['log_clv']

# Features
X = df[final_feature_cols].copy()

# Handle any remaining missing values in engineered features
for col in X.columns:
    if X[col].dtype in ['object', 'category']:
        X[col] = X[col].fillna('unknown')
    else:
        X[col] = X[col].fillna(X[col].median())

# Split: 80% train, 20% test
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

print(f"\n📊 Split Summary:")
print(f"   Training Set: {len(X_train):,} samples ({100-TEST_SIZE*100:.0f}%)")
print(f"   Test Set:     {len(X_test):,} samples ({TEST_SIZE*100:.0f}%)")
print(f"\n   Feature Dimensions: {X_train.shape[1]} features")

---

## 7. Preprocessing Pipeline

We use scikit-learn's `ColumnTransformer` to apply different transformations to different column types:

- **Numerical Features**: StandardScaler (z-score normalization)
- **Categorical Features**: OneHotEncoder (binary dummies)

In [ ]:
# Build preprocessing pipeline
print("=" * 80)
print("PREPROCESSING PIPELINE")
print("=" * 80)

# Recalculate feature types from training data
final_numeric = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
final_categorical = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n🔧 Building ColumnTransformer:")
print(f"   Numerical ({len(final_numeric)}): StandardScaler")
print(f"   Categorical ({len(final_categorical)}): OneHotEncoder")

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), final_numeric),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), final_categorical)
    ],
    remainder='passthrough'  # Keep any unspecified columns as-is
)

# Fit on training data only (to prevent data leakage)
print("\n⏳ Fitting preprocessor on training data...")
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"\n✅ Preprocessing Complete!")
print(f"   Original features: {X_train.shape[1]}")
print(f"   Transformed features: {X_train_processed.shape[1]}")

In [ ]:
# Get feature names after transformation
feature_names = preprocessor.get_feature_names_out()

print(f"\n📋 Processed Feature Names (first 20):")
for i, name in enumerate(feature_names[:20]):
    print(f"   {i+1}. {name}")
if len(feature_names) > 20:
    print(f"   ... and {len(feature_names) - 20} more features")

In [ ]:
# Convert to DataFrames for easier inspection
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names)

print("\n📊 Processed Training Data Summary:")
X_train_df.describe().T.head(10)

---

## 8. Save Artifacts

We save the preprocessed data and fitted preprocessor for use in modeling.

In [ ]:
# Save processed data
print("=" * 80)
print("SAVING ARTIFACTS")
print("=" * 80)

# Save processed feature matrices
X_train_df.to_csv(os.path.join(DATA_PROCESSED_DIR, 'X_train_processed.csv'), index=False)
X_test_df.to_csv(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.csv'), index=False)

# Save target variables
pd.Series(y_train).to_csv(os.path.join(DATA_PROCESSED_DIR, 'y_train.csv'), index=False)
pd.Series(y_test).to_csv(os.path.join(DATA_PROCESSED_DIR, 'y_test.csv'), index=False)

print(f"\n✅ Saved processed data:")
print(f"   • X_train_processed.csv ({X_train_df.shape[0]:,} × {X_train_df.shape[1]})")
print(f"   • X_test_processed.csv ({X_test_df.shape[0]:,} × {X_test_df.shape[1]})")
print(f"   • y_train.csv ({len(y_train):,} samples)")
print(f"   • y_test.csv ({len(y_test):,} samples)")

In [ ]:
# Save the fitted preprocessor
preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
joblib.dump(preprocessor, preprocessor_path)

print(f"\n✅ Saved fitted preprocessor: {preprocessor_path}")

# Save feature names
feature_names_path = os.path.join(MODELS_DIR, 'feature_names.txt')
with open(feature_names_path, 'w') as f:
    for name in feature_names:
        f.write(f"{name}\n")

print(f"✅ Saved feature names: {feature_names_path}")

In [ ]:
# Save a complete model-ready dataset (for convenience)
model_ready_df = pd.concat([
    pd.DataFrame(X_train_processed, columns=feature_names),
    pd.DataFrame({'log_clv': y_train.values})
], axis=1)

model_ready_path = os.path.join(DATA_PROCESSED_DIR, 'model_ready_data.csv')
model_ready_df.to_csv(model_ready_path, index=False)

print(f"\n✅ Saved model-ready dataset: {model_ready_path}")
print(f"   Shape: {model_ready_df.shape}")

---

## 9. Summary

### What We Accomplished

| Step | Action | Outcome |
|------|--------|---------|
| 1 | Feature Selection | Identified 21 relevant features |
| 2 | Multicollinearity Check | Verified no severe collinearity issues |
| 3 | Target Transformation | Applied log1p to reduce skewness |
| 4 | Feature Engineering | Created 5 new features |
| 5 | Train-Test Split | 80/20 split with random state 42 |
| 6 | Preprocessing | StandardScaler + OneHotEncoder pipeline |
| 7 | Artifact Export | Saved all processed data and fitted pipeline |

In [ ]:
# Final summary
print("=" * 80)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 80)

print(f"\n📊 Dataset Overview:")
print(f"   Original records: {len(df):,}")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples:     {len(X_test):,}")

print(f"\n🔧 Feature Engineering:")
print(f"   Original features:     {len(feature_cols)}")
print(f"   Engineered features:   {len(engineered_features)}")
print(f"   Final features (after encoding): {len(feature_names)}")

print(f"\n📁 Saved Artifacts:")
print(f"   • Processed training data")
print(f"   • Processed test data")
print(f"   • Fitted preprocessor (for inference)")
print(f"   • Feature names list")
print(f"   • Model-ready dataset")

print(f"\n✅ Ready for Notebook 04: Predictive Modeling!")

---

## Next Steps

In **Notebook 04: Predictive Modeling**, we will:

1. Establish a baseline model for comparison
2. Train multiple algorithms (Linear Regression, Random Forest, Gradient Boosting)
3. Perform hyperparameter tuning
4. Evaluate models using multiple metrics
5. Analyze feature importance
6. Select the best model for deployment

---

**End of Notebook 03**